In [84]:
import pandas as pd
import numpy as np

In [85]:
df = pd.read_csv("attestation_data.csv")
df = df.fillna('')
df

,Validator index,Skip,Orphan,Missed gossip,Propagation,Got in time but vc voted for previous,Double vote,Invalid local signature
0,6686,157088;152864;153248;165025;165376;,,156992;153408;169600;166432;168320;,152896;154321;168384;165988;166094;166623;,,,
1,6687,157921;165890;166144;,,157888;156992;156928;168320;169313;,165770;,166880;,,
2,6690,158016;154304;169312;169952;171296;166368;1685...,154336;,153376;153792;166176;166240;168736;,166615;166749;,,,
3,6717,153185;169216;,154323;,158176;158080;156832;156161;152608;,166886;,,,
4,6718,169952;172416;168641;,154323;,158080;157888;156576;168320;170176;167040;,,,154365;,
5,6719,157696;154336;170208;171744;172642;,154321;,157600;153728;154720;172352;,,,,
6,6720,157088;156609;153345;153600;154309;167582;1719...,,,,,,154339;
7,6721,156704;,154301;154333;,157984;155712;,,,154343;,


In [86]:
def to_list(x):
    try:
        return list(map(int, x[:-1].split(';')))
    except:
        return x

df['all'] = df.iloc[:,1:].sum(axis=1)
df['all'] = df['all'].apply(lambda x: list(map(int, str(x).split(';')[:-1])))

# The slot to be attested at was a skip slot. We voted for the previous non skip slot block
df['Skip'] = df['Skip'].apply(to_list)
# The block we voted for was orphaned.
df['Orphan'] = df['Orphan'].apply(to_list)
# We got the block for the slot after 1/4th of the slot had passed.
df['Missed gossip'] = df['Missed gossip'].apply(to_list)
# Voted for the right block still missed the attestation. Probably bad gossip peers.
df['Propagation'] = df['Propagation'].apply(to_list)
# Got the block before 1/4th of a slot passed, still voted for previous block.
df['Got in time but vc voted for previous'] = df['Got in time but vc voted for previous'].apply(to_list)
# Slashable attestation. Didn't vote
df['Double vote'] = df['Double vote'].apply(to_list)
# BN failed with invalid local signature
df['Invalid local signature'] = df['Invalid local signature'].apply(to_list)
df['all'] = df['all'].apply(to_list)
df

,Validator index,Skip,Orphan,Missed gossip,Propagation,Got in time but vc voted for previous,Double vote,Invalid local signature,all
0,6686,"[157088, 152864, 153248, 165025, 165376]",,"[156992, 153408, 169600, 166432, 168320]","[152896, 154321, 168384, 165988, 166094, 166623]",,,,"[157088, 152864, 153248, 165025, 165376, 15699..."
1,6687,"[157921, 165890, 166144]",,"[157888, 156992, 156928, 168320, 169313]",[165770],[166880],,,"[157921, 165890, 166144, 157888, 156992, 15692..."
2,6690,"[158016, 154304, 169312, 169952, 171296, 16636...",[154336],"[153376, 153792, 166176, 166240, 168736]","[166615, 166749]",,,,"[158016, 154304, 169312, 169952, 171296, 16636..."
3,6717,"[153185, 169216]",[154323],"[158176, 158080, 156832, 156161, 152608]",[166886],,,,"[153185, 169216, 154323, 158176, 158080, 15683..."
4,6718,"[169952, 172416, 168641]",[154323],"[158080, 157888, 156576, 168320, 170176, 167040]",,,[154365],,"[169952, 172416, 168641, 154323, 158080, 15788..."
5,6719,"[157696, 154336, 170208, 171744, 172642]",[154321],"[157600, 153728, 154720, 172352]",,,,,"[157696, 154336, 170208, 171744, 172642, 15432..."
6,6720,"[157088, 156609, 153345, 153600, 154309, 16758...",,,,,,[154339],"[157088, 156609, 153345, 153600, 154309, 16758..."
7,6721,[156704],"[154301, 154333]","[157984, 155712]",,,[154343],,"[156704, 154301, 154333, 157984, 155712, 154343]"


In [90]:
all_slots = list()
for slots in df['all']:
    all_slots.extend(slots)

print("Min slot", min(all_slots))
print("Max slot", max(all_slots))
print("Range", max(all_slots) - min(all_slots))

Min slot 152608
Max slot 172642
Range 20034


In [91]:
# Get distribution of missed attestations over slot number in epoch.
# e.g. {0: 10} indicates that there are 10 missed attestations for the given key type at slot 0 in an epoch 
def distribution(key):    
    all_slots = list()
    for slots in df[key]:
        all_slots.extend(slots)
    d = dict()
    for slot in all_slots:
        try:
            d[slot % 32] +=1
        except:
            d[slot % 32] = 1
    print("Miss type: ", key)
    print("Distribution", d)
    print("Number of unique missed slots",len(set(all_slots)))
    print("Number of missed slots", len(all_slots))
    print('\n')

distribution('Skip')
distribution('Orphan')
distribution('Missed gossip')
distribution('Propagation')
distribution('all')

Miss type:  Skip
Distribution {0: 25, 1: 6, 2: 2, 5: 1, 30: 1}
Number of unique missed slots 33
Number of missed slots 35


Miss type:  Orphan
Distribution {0: 1, 19: 2, 17: 1, 29: 2}
Number of unique missed slots 5
Number of missed slots 6


Miss type:  Missed gossip
Distribution {0: 30, 1: 2}
Number of unique missed slots 27
Number of missed slots 32


Miss type:  Propagation
Distribution {0: 2, 17: 1, 4: 1, 14: 1, 31: 1, 10: 1, 23: 1, 29: 1, 6: 1}
Number of unique missed slots 10
Number of missed slots 10


Miss type:  all
Distribution {0: 59, 1: 8, 17: 2, 4: 1, 14: 1, 31: 1, 2: 2, 10: 1, 23: 1, 29: 4, 19: 2, 6: 1, 5: 1, 30: 1, 3: 1, 7: 1}
Number of unique missed slots 77
Number of missed slots 87


